## Answering Business Questions Uing SQL

Chinook is a fictional record store that is looking to improve sales and are looking for answers to several questions.
- A new record label deal allows them to choose 3 of 4 albums to add to their store. Each album are from artists of different genres, which albums should they add to their catalog and advertise to an American audience? 
- Employees are very customer facing, with each customer assigned to a sales employee. How is employee performance, how do they differ?

First lets Connect Jupyter Notebook to database file chinook.db

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

Now we will look at the database contents

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


The database contains several tables, structred as:
![image](https://s3.amazonaws.com/dq-content/189/chinook-schema.svg)

### 1. Track Sales by Genre

To evaluate which of the 4 albums will produce the best sales, we will look at which genres of music had the highest sales in the USA. 

| Artist Name | Genre |
| --- | -: |
| Regal | Hip-Hop |
| Red Tone | Punk |
| Meteor and the Girls | Pop |
| Slim Jim Bites | Blues|

Below is a table with the total number of tracks sold, and the percentage of sales for the top 10 genres in the USA.

In [3]:
%%sql 
WITH usa_track_sales as (
        SELECT 
              inv_l.quantity as quantity,
              g.name as genre
        FROM customer as c 
        INNER JOIN invoice as inv ON c.customer_id = inv.customer_id
        INNER JOIN invoice_line as inv_l ON inv_l.invoice_id = inv.invoice_id
        INNER JOIN track as t ON t.track_id = inv_l.track_id 
        INNER JOIN genre as g ON g.genre_id = t.genre_id
        WHERE country = "USA"
     )

SELECT 
     genre,
     SUM(quantity) as quantity_sold,
     ROUND(CAST(SUM(quantity) as FLOAT) / (
                                           SELECT COUNT(*) 
                                           FROM usa_track_sales
                                          ) * 100, 2) 
                                          as percentage_sales
     FROM usa_track_sales
     GROUP BY genre
     ORDER BY quantity_sold DESC
     LIMIT 10

Done.


genre,quantity_sold,percentage_sales
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


Based on track sale performance by genre, we would select the albums by
- Red Tone (Punk)
- Slim Jim Bites (Blues)
- Meteor and the Girls (Pop)

It should be noted that the difference in sales observed in our data between Pop and Hip-Hop are very small. In addition, the sales from Punk, Blues, and Pop tracks combined only accounts for about 17.89% of total track sales in the USA. There is a strong preference for the rock genre, making up over 50% of total sales, Chinook should look to add tracks from that genre.

### 2. Employee Sales

To examine the performance of sale support agents, we will look at each employees accumlated sales from their hire date.

In [4]:
%%sql 
WITH sales_agents AS (
    SELECT 
          c.customer_id,
          e.employee_id,
          e.first_name,
          e.last_name,
          e.hire_date,
          i.total
    FROM customer as c
    INNER JOIN employee as e ON e.employee_id = c.support_rep_id
    INNER JOIN invoice as i ON i.customer_id = c.customer_id
)

SELECT 
     first_name || " " || last_name as name,
     hire_date,
     ROUND(SUM(total), 0) as total_sales,
     ROUND(SUM(total) / (
                         SELECT SUM(total) 
                         FROM sales_agents
                        ) 
           * 100, 2) as sale_percentage
FROM sales_agents
GROUP BY employee_id

Done.


name,hire_date,total_sales,sale_percentage
Jane Peacock,2017-04-01 00:00:00,1732.0,36.77
Margaret Park,2017-05-03 00:00:00,1584.0,33.63
Steve Johnson,2017-10-17 00:00:00,1394.0,29.6


From the table above, we see a 7% difference in total sales between the highest and lowest performing employee. However, the differences in total sales and sales percentage can be explained by their hire dates and we don't observe any large differences in performance.

### 3. Sales Data By Country

We are interesed in the total number of customers in each country, their total value of sales, the average amount a customer spends in each country, and the average value in each order. Countires with only one customer will be combined into an *Other* category. 

In [5]:
%%sql
WITH other_country AS (
    SELECT 
        c.country as all_countries,
        COUNT(DISTINCT c.customer_id) as number_customers,
        SUM(i.total) as total_sales,
        SUM(i.total)/COUNT(DISTINCT c.customer_id) as avg_customer_sales,
        SUM(i.total)/COUNT(DISTINCT i.invoice_id) as avg_order_total,
        CASE 
            WHEN COUNT(DISTINCT c.customer_id) = 1
            THEN 1000
            ELSE -COUNT(DISTINCT c.customer_id)
        END as sort
    FROM customer as c
    INNER JOIN invoice as i ON c.customer_id = i.customer_id
    GROUP BY all_countries
)

SELECT 
    CASE 
        WHEN number_customers = 1 
        THEN "Other" 
    ELSE all_countries
    END as country,
    SUM(number_customers) as customers,
    ROUND(SUM(total_sales), 2) as total_sales,
    ROUND(AVG(avg_customer_sales), 2) as avg_customer_sales,
    ROUND(AVG(avg_order_total), 2) as avg_order_total
FROM other_country
GROUP BY country
ORDER BY sort;


Done.


country,customers,total_sales,avg_customer_sales,avg_order_total
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
United Kingdom,3,245.52,81.84,8.77
Czech Republic,2,273.24,136.62,9.11
India,2,183.15,91.57,8.72
Portugal,2,185.13,92.57,6.38
Other,15,1094.94,73.0,7.45


<p> From our data, we can see that Unitied States is currently the market we obtain most of our customers. If we don't want to spend extra on acquring customers from other countries, new tracks added to the store should be tailored to customers in USA and Canada as they make up a large portion of our customer base,  </p>
<p> Countries we may want to target in the future are Czech Republic, India, and the United Kingdom with the highest average spend per order. Do note that our dataset is very small and what is observed could be due to chance. More data should be collected in the targeted countries to determine if what we observe here is still true with more customers. In addition to data on other factors, such as costs associated with acquiring new customers in different countries.</p>

### 4. Albums vs Individual Track Sales

Management is considering a new strategy to cut down on their costs, however are worried how this strategy may impact the companies revenue. Currently, Chinook purchases entire albums from record companies, paying for every track in the album. Instead of this, they would like to only purchase the popular hits that produce the highest sale volumes to add to the store. To help them answer this, we will examine the percentage of sales for individual tracks vs whole albums. 

In [70]:
%%sql
/* invoice_line table has information on invoice_id and track_id, combined with the track table we can see which track belongs to which album*/
WITH invln_tracks AS(
    SELECT 
        invln.invoice_id,
        invln.track_id,
        t.album_id
        FROM invoice_line as invln
        LEFT JOIN track as t ON t.track_id = invln.track_id
), 

/* calculate the number of tracks in each album from the track table */
album_length AS(
    SELECT 
        album_id as album,
        COUNT(DISTINCT track_id) as length 
    FROM track
    GROUP BY album_id
), 

/* then calculate the number of distinct tracks purcahsed from each album, in every invoice */
album_purchases AS(
    SELECT 
        invln_t.invoice_id,
        invln_t.album_id,
        COUNT(DISTINCT invln_t.track_id) as album_tracks_purchased,
        album_length.length,
        CASE
            WHEN album_length.length = COUNT(DISTINCT invln_t.track_id) 
                AND album_length.length > 2
            THEN 1
            ELSE 0 
        END as full_album
    FROM invln_tracks as invln_t
    LEFT OUTER JOIN album_length ON invln_t.album_id = album_length.album
    GROUP BY invoice_id, album_id 
    ORDER BY invoice_id, full_album
)

SELECT 
    purchased_album,
    COUNT(invoice_id) as number_invoices,
    ROUND(COUNT(invoice_id)/CAST((SELECT COUNT(DISTINCT invoice_id)
                                  FROM album_purchases) 
                            as FLOAT) * 100, 2) as percent_invoices
FROM (SELECT 
          invoice_id, 
          MAX(full_album) as full_album, 
          CASE 
              WHEN MAX(full_album) = 1   /* When the number of tracks purchased in the invoice from the album matches total number of tracks in the album, flag 1 */
              THEN "yes"
              ELSE "no"
          END AS purchased_album
      FROM album_purchases
      GROUP BY invoice_ID)
GROUP BY purchased_album

Done.


purchased_album,number_invoices,percent_invoices
no,503,81.92
yes,111,18.08


We observe that about 18% of total invoices include a purchse of a full album. Due to this being a good proportion of invoices, only purchasing popular tracks from record labels instead of full albums will likely hurt the companies revenue. Therefore we advise against this new strategy.